# Knowledge Distillation with BERT

## Overview
An advanced implementation of knowledge distillation using a custom distillation loss for sentiment classification.

In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
from transformers import BertForSequenceClassification, BertTokenizer, BertModel
from datasets import load_dataset
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Import the custom distillation loss
import sys
sys.path.append('/app/src')
from loss.distillation import DistillationLoss

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
# Load dataset
dataset = load_dataset('glue', 'sst2')['train']

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and prepare data
def prepare_data(examples, max_length=128):
    encodings = tokenizer(
        examples['sentence'], 
        truncation=True, 
        padding='max_length', 
        max_length=max_length
    )
    
    return {
        'input_ids': torch.tensor(encodings['input_ids']),
        'attention_mask': torch.tensor(encodings['attention_mask']),
        'labels': torch.tensor(examples['label'])
    }

# Prepare dataset
processed_data = prepare_data(dataset[:1000])  # Use first 1000 samples for demonstration

In [3]:
# Custom Student Model with Reduced Complexity
class StudentBertModel(nn.Module):
    def __init__(self, num_labels=2, hidden_size=768, num_hidden_layers=6):
        super(StudentBertModel, self).__init__()
        
        # Base BERT model with reduced layers
        self.bert = BertModel.from_pretrained('bert-base-uncased', 
                                               num_hidden_layers=num_hidden_layers)
        
        # Simplified classification head
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size // 2, num_labels)
        )
        
    def forward(self, input_ids, attention_mask):
        # Get BERT outputs
        bert_outputs = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask
        )
        
        # Use the pooled output for classification
        pooled_output = bert_outputs.pooler_output
        logits = self.classifier(pooled_output)
        
        return logits
    
    def get_attention_and_values(self, input_ids, attention_mask):
        # Get BERT outputs including attention
        bert_outputs = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask,
            output_attentions=True
        )
        
        # Return attention and values
        return (
            bert_outputs.attentions,  # Attention weights
            bert_outputs.last_hidden_state  # Value vectors
        )

In [4]:
# Training Hyperparameters
batch_size = 32
num_epochs = 100
learning_rate = 2e-5

# Create DataLoader
dataset = TensorDataset(
    processed_data['input_ids'], 
    processed_data['attention_mask'], 
    processed_data['labels']
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize teacher and student models
teacher_model = BertForSequenceClassification.from_pretrained('bert-base-uncased').to(device)
teacher_base_model = teacher_model.bert  # Extract the base BERT model
student_model = StudentBertModel(num_labels=2).to(device)

# Freeze teacher model
for param in teacher_model.parameters():
    param.requires_grad = False

# Training Setup
optimizer = torch.optim.AdamW(student_model.parameters(), lr=2e-5)
distillation_criterion = DistillationLoss(temperature=2.0).to(device)
classification_criterion = nn.CrossEntropyLoss()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Training Loop
for epoch in range(num_epochs):
    student_model.train()
    teacher_model.eval()
    total_loss = 0
    
    progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for batch in progress_bar:
        # Unpack batch
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass for teacher
        with torch.no_grad():
            teacher_output = teacher_model(input_ids, attention_mask=attention_mask)
            teacher_bert_output = teacher_base_model(
                input_ids, 
                attention_mask=attention_mask, 
                output_attentions=True
            )
            teacher_attention = teacher_bert_output.attentions
            teacher_values = teacher_bert_output.last_hidden_state
        
        # Forward pass for student
        student_logits = student_model(input_ids, attention_mask)
        student_attention, student_values = student_model.get_attention_and_values(input_ids, attention_mask)
        
        # Compute distillation loss
        # Aggregate attention and values across layers
        student_A = torch.mean(torch.stack(student_attention), dim=0)
        teacher_A = torch.mean(torch.stack(teacher_attention), dim=0)
        
        knowledge_loss = distillation_criterion(
            teacher_A, teacher_values,
            student_A, student_values
        )
        
        # Compute classification loss
        classification_loss = classification_criterion(student_logits, labels)
        
        # Combine losses
        total_batch_loss = knowledge_loss
        
        # Backward pass
        total_batch_loss.backward()
        optimizer.step()
        
        total_loss += total_batch_loss.item()
        progress_bar.set_description(f'Epoch {epoch+1}/{num_epochs}')
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}')

# Save student model
torch.save(student_model.state_dict(), './saved_models/student_model.pt')

Epoch 1/100:   0%|          | 0/32 [00:00<?, ?it/s]BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Epoch 1/100, Loss: 46.6790


Epoch 2/100, Loss: 25.1429


Epoch 3/100, Loss: 19.3854


Epoch 4/100, Loss: 17.0347


...

Epoch 97/100, Loss: 2.8747


Epoch 98/100, Loss: 2.8997


Epoch 99/100, Loss: 2.8340


Epoch 100/100, Loss: 2.8285


RuntimeError: Parent directory ./saved_models does not exist.

In [6]:
torch.save(student_model.state_dict(), 'student_model.pt')
